In [1]:
import sys

sys.path.append('../..')

The first step is to prepare the dataset in the pytorch Dataset format.
For example like this with COCO annotation format:

In [2]:
from torchvision.transforms import ToTensor

from core.architecture.datasets.object_detection_datasets import COCODataset

train_dataset_path = '/media/n31v/data/datasets/ALET/train' # your path to train part of dataset
val_dataset_path = '/media/n31v/data/datasets/ALET/val' # your path to validation part of dataset
train_json_path = '/media/n31v/data/datasets/ALET/train.json' # your path to train annotations of dataset
val_json_path = '/media/n31v/data/datasets/ALET/val.json' # your path to validation annotations of dataset

transform = ToTensor()
train_dataset = COCODataset(images_path=train_dataset_path, json_path=train_json_path, transform=transform)
val_dataset = COCODataset(images_path=val_dataset_path, json_path=val_json_path, transform=transform)

100%|██████████| 2191/2191 [00:00<00:00, 421527.46it/s]


The second step is model initialization.

In [3]:
from core.architecture.experiment.nn_experimenter import FasterRCNNExperimenter

experimenter = FasterRCNNExperimenter(
    num_classes = len(train_dataset.classes) + 1,
    model_params={'weights': 'DEFAULT'}
)

The third step (optionally) initialization of the structure optimization.

In [4]:
from core.operation.optimization.structure_optimization import SVDOptimization

svd_optimization = SVDOptimization(energy_thresholds=[0.1, 0.3, 0.5, 0.7, 0.9, 0.99, 0.995, 0.999, 1])

Running Model Training.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import warnings
warnings.filterwarnings("ignore")

experimenter.fit(
    dataset_name='ALET', # Name of your dataset
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    num_epochs=50,
    structure_optimization=svd_optimization, # Optional
    dataloader_params={
        'batch_size': 2,
        'num_workers': 2,
    }
)

Now you can get predictions for all images in a folder. The method returns a dictionary {'image name': predictions}

In [ ]:
experimenter.predict('') # Path to your folder with images